# Progetto — Previsione del carico elettrico

In questo progetto affrontiamo un caso reale di **previsione del carico elettrico**.

I dati utilizzati provengono dal **Download Center di Terna**  
https://dati.terna.it/en/download-center  

In particolare, useremo i dati di **carico elettrico della zona Nord** relativi agli anni **2024 e 2025**.  
Nei dataset Terna, il carico elettrico rappresenta la **potenza richiesta al sistema elettrico in ciascun intervallo temporale** ed è una grandezza centrale per la pianificazione e la gestione operativa della rete di trasmissione.

L’obiettivo dell’esercizio è utilizzare queste osservazioni storiche per **prevedere l’andamento del carico nei primi giorni del 2026**, riproducendo in forma semplificata un problema tipico affrontato dagli operatori di sistema.

Accanto alla serie storica del carico, integreremo **dati di temperatura** ottenuti tramite API da Open-Meteo  
https://open-meteo.com  

La temperatura verrà utilizzata come **variabile esogena** a supporto della previsione, per mostrare come informazioni esterne possano migliorare un modello di forecasting.

# STEP 1: Caricamento dei dati e unione

In questo step costruiamo una singola serie storica continua a partire dai dati grezzi.

1. Leggi i due file Excel presenti nella cartella `Dati/`:
   - `load_total_north_hourly_2024.xlsx`
   - `load_total_north_hourly_2025.xlsx`

2. Unisci i due dataset **per righe**, mantenendo lo stesso schema di colonne, in modo da ottenere una serie unica che copra l’intero periodo 2024–2025.

In [ ]:
# fai l'import di pandas

# Carica i dati dei due anni (df_2024 e df_2025)

# Unisci i due dataset per righe (pd.concat con ignore_index=True)

# STEP 2: Preparazione della colonna temporale e prima visualizzazione

In questo step rendiamo la colonna data utilizzabile come variabile temporale e facciamo una prima ispezione visiva della serie.

1. Identifica la colonna che contiene la data/ora e trasformala in formato `datetime`.
   - Se la data è letta come stringa, la convertiamo con `pd.to_datetime(...)`.

2. Ordina il dataset per data (anche se sembra già ordinato) e reimposta l’indice per avere un ordinamento pulito.

3. Importa Plotly e visualizza **entrambe** le serie nel tempo:
   - `Total Load [MW]`
   - `Forecast Total Load [MW]`

4. Dopo aver verificato che le due curve siano coerenti, elimina le colonne che non useremo nel forecasting:
   - `Forecast Total Load [MW]`
   - `Bidding Zone`

Al termine di questo step deve rimanere una serie storica con:
- una colonna temporale in formato `datetime`;
- una sola colonna target (`Total Load [MW]`), pronta per essere rinominata in `y` più avanti.


In [ ]:
import plotly.express as px

# Converti la colonna Date in formato datetime

# Ordina per data 

# resetta l'indice (df = df.reset_index(drop=True))

# Elimina le colonne non necessarie ( Forecast Total Load [MW] e Bidding Zone )

# Visualizza Total Load

# STEP 3: Aggregazioni temporali e profili medi

In questo step analizziamo la serie oraria cambiando punto di vista sul tempo.  
Costruiamo una serie giornaliera e osserviamo pattern medi legati all’ora del giorno e al giorno della settimana.

Useremo:
- `Date` come colonna temporale;
- `Total Load [MW]` come variabile di interesse.

---

## 3.1 Serie giornaliera (somma)

Esegui i passaggi seguenti nell’ordine:

1. Parti dal DataFrame orario `df`.

2. Imposta la colonna `Date` come indice temporale.

3. Aggrega per giorno usando `resample("D")` e calcola la **somma** del carico.

4. Riporta `Date` come colonna e salva il risultato in `df_daily`.

5. Visualizza la serie giornaliera con Plotly.



In [ ]:
# =============================================================================
# 3.1 Serie giornaliera (somma)
# =============================================================================

# 1-2. Imposta Date come indice

# 3. Aggrega per giorno (somma) hint: resample come visto nel notebook 06
df_daily = ...

# 4. Riporta Date come colonna (reset index) - completo: basta rimuovere il commento
# df_daily = df_daily.reset_index()

# 5. Visualizza la serie giornaliera


---

## 3.2 Profilo medio giornaliero (media per ora del giorno)

Ora torniamo alla serie oraria originale.

6. Estrai l’**ora del giorno** dalla colonna `Date` e salvala in una nuova colonna `hour`.

7. Raggruppa per `hour` e calcola la **media** del carico.

8. Visualizza il profilo medio giornaliero con Plotly.

In [ ]:

# =============================================================================
# 3.2 Profilo medio giornaliero (media per ora del giorno)
# =============================================================================

# 6. Estrai l'ora del giorno (usa dt e hour) come in notebook 06

# 7. Raggruppa per hour e calcola la media (hint: groupby, mean, reset_index)
hourly_profile = ...

# 8. Visualizza il profilo medio giornaliero

---

## 3.3 Profilo medio settimanale (media per giorno della settimana)

Sempre partendo dalla serie oraria originale:

9. Estrai il **giorno della settimana** dalla colonna `Date` e salvalo in una nuova colonna `weekday`
   - usa la convenzione: `0 = Lunedì, …, 6 = Domenica`.

10. Raggruppa per `weekday` e calcola la **media** del carico.

11. Visualizza il profilo medio settimanale con Plotly.

In [ ]:
# =============================================================================
# 3.3 Profilo medio settimanale (media per giorno della settimana)
# =============================================================================

# 9. Estrai il giorno della settimana (0=Lunedì, 6=Domenica) - hint: dt e dayofweek
df['weekday'] = ...

# 10. Raggruppa per weekday e calcola la media (hint: groupby, mean, reset_index)

# Aggiungi etichette leggibili per i giorni (completo: basta rimuovere il commento)
#weekday_names = ['Lunedì', 'Martedì', 'Mercoledì', 'Giovedì', 'Venerdì', 'Sabato', 'Domenica']
#weekly_profile['day_name'] = weekly_profile['weekday'].map(lambda x: weekday_names[x])

# 11. Visualizza il profilo medio settimanale (line plot)

# .. completa qui ...
#fig.update_xaxes(categoryorder='array', categoryarray=weekday_names)
#fig.show()

# STEP 4: Primo modello di forecasting con Prophet (univariato)

In questo step costruiamo il **primo modello di forecasting** utilizzando esclusivamente la serie storica del carico elettrico.

L’obiettivo è:
- preparare i dati nel formato richiesto da Prophet;
- addestrare un modello base;
- generare una previsione su un orizzonte temporale realistico;
- osservare sia la previsione sia le componenti stimate dal modello.

---

## 4.1 Preparazione del dataset per Prophet

Prophet richiede un DataFrame con **due sole colonne**:
- `ds`: variabile temporale (`datetime`);
- `y`: variabile da prevedere.

Esegui i passaggi seguenti:

1. Seleziona dal DataFrame originale solo le colonne:
   - `Date`
   - `Total Load [MW]`

2. Rinominale rispettivamente in:
   - `ds`
   - `y`

3. Verifica che:
   - `ds` sia in formato `datetime`;
   - non ci siano valori mancanti;
   - le osservazioni siano ordinate nel tempo.

---
## 4.2 Inizializzazione e addestramento del modello

Ora inizializziamo un modello Prophet **standard**, senza configurazioni avanzate.

1. Crea un’istanza del modello.
2. Addestra il modello sui dati storici usando `fit`.

In questa fase Prophet:
- analizza il trend;
- stima le stagionalità;
- apprende la struttura della serie dal passato.


In [ ]:
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

# -------------------------------------------------
# Preparazione del dataset per Prophet (univariato)
# -------------------------------------------------
df_prophet = ...

df_prophet = df_prophet.reset_index(drop=True)

# -------------------------------------------------
# Inizializzazione e fit del modello
# -------------------------------------------------

# ... completa qui ...

# -------------------------------------------------
# Costruzione del periodo futuro e previsione
# Orizzonte: 7 giorni (orario)
# -------------------------------------------------
# hint: make_future_dataframe (7 giorni: quanti 15 min?)

# hint: predict 

# -------------------------------------------------
# Visualizzazione previsione e componenti
# -------------------------------------------------
# hint: plot_plotly , plot_components_plotly



# STEP 5: Recupero e integrazione della temperatura (Open-Meteo)

In questo step aggiungiamo una variabile esogena: la **temperatura a 2 metri** misurata/ricostruita per Milano.
È una semplificazione: usiamo Milano come proxy “medio” per il Nord Italia.

Useremo l’API **Historical Weather** di Open-Meteo:
- non serve alcuna chiave API;
- possiamo chiedere i dati **orari** su un intervallo di date.

Documentazione:
- Historical Weather API: https://open-meteo.com/en/docs/historical-weather-api
- Home / overview: https://open-meteo.com/


In [ ]:
# Cella completa: leggete bene il codice
import requests
import pandas as pd

# intervallo dal dataset
start_date = df['Date'].min().strftime('%Y-%m-%d')
end_date = df['Date'].max().strftime('%Y-%m-%d') 
print('Intervallo:', start_date, '->', end_date)

# Milano (punto rappresentativo Nord Italia per esercizio)
lat, lon = 45.4642, 9.1900

# Open-Meteo – storico orario
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": lat,
    "longitude": lon,
    "start_date": start_date,
    "end_date": end_date,
    "hourly": "temperature_2m",
    "timezone": "Europe/Rome" # "UTC"
}

r = requests.get(url, params=params, timeout=20)
r.raise_for_status()

# DataFrame temperatura
hourly = r.json()["hourly"]
w = pd.DataFrame(hourly)

In [ ]:
w["time"] = pd.to_datetime(w["time"])
w = w.rename(columns={
    "time": "Date",
    "temperature_2m": "Temperature_C"
})
w["Date"] = pd.to_datetime(w["Date"])
w = w.set_index("Date").sort_index()

Ora dobbiamo integrare il dato di temperatura con il dato di load. Per prima cosa va risolto la differente granularità, visto che il dato medio lo abbiamo solo orario (upsampling h -> 15min)

    1. fare resample a quartorario per poter integrare con load (usa resemple + interpolate con method linear) e salva il risultato in w_15
    
    2. merge con df load

In [ ]:
# upsample a 15 minuti con interpolazione lineare (hint: resample , interpolate )
w_15 = ...
#w_15.reset_index(inplace=True) 

In [ ]:
# merge con il DataFrame load
df_full = pd.merge(
    ... , # dataframe di sinistra (df) con il load 
    ... , # dataframe di destra (w_15) con la temperatura
    on=..., # Colonna comune su cui fare la join (Date)
    how='left' 
)

# how left -> left join (tiene tutti i record di df, aggiunge temperatura se presente)
# perché temperatura non ha i valori dell'ultima ora dell'anno (verifica con .tail() di df e w_15)
# ... completa qui ...

In [ ]:
# mancano i dati dell'ultima ora del 2025, usiamo ffill
df_full['Temperature_C'] = df_full['Temperature_C'].ffill()
# verifica che non ci siano più NaN con .tail()

In [ ]:
# visualizza la temperatura con px.line()
# ...

# STEP 6: Forecast con temperatura (Prophet + regressore)

In questo step facciamo una previsione usando Prophet con la temperatura come variabile esogena.
La serie del carico è **quartoraria**, quindi anche il future dataset e la temperatura devono essere allineati a **15 minuti**.

Obiettivo:
- costruire `future` a 15 minuti;
- recuperare la **temperatura di previsione** da Open-Meteo sullo stesso periodo;
- allinearla ai timestamp del future dataset;
- calcolare forecast e visualizzare forecast + componenti.

Fonte API (forecast): https://open-meteo.com/en/docs


In [ ]:
from prophet import Prophet

# 1) Selezione e rinomina (Prophet vuole ds, y + regressori)
df_prophet = ...
#df_prophet = df_prophet.reset_index(drop=True)

# 2) Inizializzazione modello + regressore (.add_regressor come visto nel notebook 07)
#m = ...
#m.

# 3) Fit
# ...

In [ ]:
import requests
import pandas as pd
from prophet.plot import plot_plotly, plot_components_plotly

# -----------------------
# 1) Future dates (3 giorni quartorari)
# -----------------------
future = m.make_future_dataframe(periods=..., freq="15min")

# -----------------------
# 2) Temperatura futura da Open-Meteo (Forecast API)
# -----------------------
lat, lon = 45.4642, 9.1900  # Milano
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": lat,
    "longitude": lon,
    "hourly": "temperature_2m",
    "timezone": "Europe/Rome",
    "start_date": future['ds'].min().strftime('%Y-%m-%d'),
    "end_date": future['ds'].max().strftime('%Y-%m-%d'),
    "timezone": "Europe/Rome" # "UTC"

}

r = requests.get(url, params=params, timeout=30)
r.raise_for_status()
data = r.json()["hourly"]

df_temp_future = pd.DataFrame({
    "ds": pd.to_datetime(data["time"]),
    "temperature": data["temperature_2m"],
})


In [ ]:
# -----------------------
# 3) Merge: future + temperatura
# -----------------------
future = future.merge(df_temp_future, on="ds", how="left").sort_values("ds")
# In caso di piccoli buchi (può capitare per edge-case), riempiamo in modo semplice
future["temperature"] = future["temperature"].ffill().bfill()

# -----------------------
# 4) Previsione + plot
# -----------------------
# ...